In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/Shareddrives/Computational Semantics A3/Code'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/Computational Semantics A3/Code


In [ ]:
import os
import spacy
import gensim.downloader as api
from gensim.models import Word2Vec
import string
import numpy as np

In [ ]:
def sentence_segmentation(text, spacy_pipeline:str="en_core_web_lg"):
  try:
      nlp = spacy.load(spacy_pipeline)
  except:
      !python3 -m spacy download {spacy_pipeline}
      nlp = spacy.load(spacy_pipeline)


  doc = nlp(text)
  assert doc.has_annotation("SENT_START")
  sentences = [sentence.text for sentence in doc.sents]
  return sentences

def get_relevant_files(decade:int):
  files = []
  for (dirpath, dirnames, filenames) in os.walk('coha_samples_text'):
    # only read the files within the selected decade
    files = ['coha_samples_text/'+f for f in filenames if int(f.split(".")[0].split("_")[1]) >= decade and int(f.split(".")[0].split("_")[1]) <= decade+10]

  return files

def extract_text(file_path:str):
  with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    text = text.lower()

  return text

def process_text(text:str, output_path:str):
  output = ""

  if os.path.exists(output_path):
    return
  else:
    with open(output_path, 'w') as f:
      f.write(output)

  if len(text) > 1000000:
    sentences = []
    for i in range(0, len(text), 1000000):
      sentences.extend(sentence_segmentation(text[i:i+1000000]))
  else:
    sentences = sentence_segmentation(text)

  output = "\n".join(sentences)

  with open(output_path, 'a') as f:
    f.write(output)


In [ ]:
for decade in range(1860, 2010, 10):
  output_path = f"coha_processed_text/{decade}.txt"
  if not os.path.exists(output_path):
    with open(output_path, 'w') as f:
      f.write("")

  files = get_relevant_files(decade)

  for f in files:
    text = extract_text(f)
    with open(output_path, 'a') as f:
      f.write(text)

In [ ]:
for decade in range(1860, 2010, 10):
  input_path = f"coha_processed_text/{decade}.txt"
  output_path = f"coha_processed_text/{decade}_processed.txt"

  text = extract_text(input_path)
  process_text(text, output_path)

In [ ]:
from gensim.models import Word2Vec

for decade in range(1860, 2010, 10):
  model = Word2Vec(min_count=50,max_vocab_size=50000)
  model.build_vocab(corpus_file=f'coha_processed_text/{decade}_processed.txt')
  total_words = model.corpus_total_words
  model.train(corpus_file=f'coha_processed_text/{decade}_processed.txt', total_words=total_words, epochs=5)
  model.save(f'coha_decade_models/{decade}.model')